# Normal Day Analysis

In [49]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

In [50]:
testfile='test7.parquet'

In [51]:
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.


In [52]:
test=pd.read_parquet(testfile)

In [53]:
test.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
781742,1550453,192.168.107.134,192.168.107.227,udp,53,199,474
781743,1550545,192.168.107.134,142.250.184.174,tcp,443,10327,59149
781744,1550546,192.168.107.134,142.250.184.174,tcp,443,9284,58110
781745,1550616,192.168.107.134,142.250.184.174,tcp,443,7955,71231
781746,1550760,192.168.107.134,142.250.184.174,tcp,443,6106,79662


In [54]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_test = test[test.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET,axis=1)]

private_dst_test.head()


,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
781742,1550453,192.168.107.134,192.168.107.227,udp,53,199,474
781761,1555264,192.168.107.134,192.168.107.234,udp,53,190,389
781762,1555268,192.168.107.134,192.168.107.234,udp,53,194,388
781763,1555274,192.168.107.134,192.168.107.234,udp,53,169,395
781767,1560348,192.168.107.134,192.168.107.234,udp,53,121,317


In [55]:
grp_private_test = private_dst_test.groupby(["proto","port"])['dst_ip'].apply(list)

In [56]:

https_servers = set(grp_private_test[('tcp', 443)])
print()

dns_servers =  set(grp_private_test[('udp', 53)])
print()

## Servers 
tcp 443 {'192.168.107.223', '192.168.107.238', '192.168.107.222', '192.168.107.226'} https

udp 53 {'192.168.107.227', '192.168.107.234'} dns


In [57]:
grouped = test.groupby(['src_ip','dst_ip']).sum()

# Calculate the ratio
grouped['ratio'] = grouped['up_bytes'] / grouped['down_bytes']
grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()

timestamp  \
src_ip          dst_ip                       
192.168.107.100 104.16.249.130   141488093   
                104.22.48.100     72663611   
                104.244.42.129   819390937   
                104.244.47.87     26913323   
                104.26.13.207     67074464   
...                                    ...   
192.168.107.99  88.157.217.145  1072668469   
                88.157.217.146   350271422   
                88.157.217.148    56185240   
                88.221.64.44     135698478   
                97.107.133.118    25547903   

                                                                            proto  \
src_ip          dst_ip                                                              
192.168.107.100 104.16.249.130  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.22.48.100   tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.244.42.129  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.244.47.87                         tcptcptcptcptcptcptcptcptcp   
                104.26.13.207   tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
...                                                                           ...   
192.168.107.99  88.157.217.145  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                88.157.217.146  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                88.157.217.148                           tcptcptcptcptcptcptcptcp   
                88.221.64.44    tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                97.107.133.118                                 tcptcptcptcptcptcp   

                                 port  up_bytes  down_bytes     ratio  
src_ip          dst_ip                                                 
192.168.107.100 104.16.249.130  18606    416652     5329752  0.078175  
                104.22.48.100   11518    289050     2517956  0.114795  
                104.244.42.129  97460   2471399    23994481  0.102999  
                104.244.47.87    3987    126016     1111322  0.113393  
                104.26.13.207    7974    208827     3153790  0.066215  
...                               ...       ...         ...       ...  
192.168.107.99  88.157.217.145  81955   2121021    19832417  0.106947  
                88.157.217.146  23922    658427     5427966  0.121303  
                88.157.217.148   3544     81995      749854  0.109348  
                88.221.64.44     9303    220648     1845226  0.119578  
                97.107.133.118   2658     56156      476165  0.117934  

[26373 rows x 6 columns]

In [58]:
mean_ratios = grouped.groupby('src_ip')['ratio'].mean().reset_index()

In [59]:
server_traffic = test[test.apply(lambda x: x['dst_ip'] in dns_servers or x['dst_ip'] in https_servers,axis=1)]


In [60]:
server_traffic.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
781742,1550453,192.168.107.134,192.168.107.227,udp,53,199,474
781761,1555264,192.168.107.134,192.168.107.234,udp,53,190,389
781762,1555268,192.168.107.134,192.168.107.234,udp,53,194,388
781763,1555274,192.168.107.134,192.168.107.234,udp,53,169,395
781767,1560348,192.168.107.134,192.168.107.234,udp,53,121,317


In [61]:
servers_grouped = server_traffic.groupby(['dst_ip','src_ip']).sum()

# Calculate the ratio
servers_grouped['ratio'] = servers_grouped['up_bytes'] / servers_grouped['down_bytes']
servers_grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()
servers_mean_ratios = servers_grouped.groupby('dst_ip')['ratio'].mean().reset_index()

In [62]:
servers_mean_ratios.sort_values(by="ratio",ascending=False)


,dst_ip,ratio
2,192.168.107.205,1.023619
0,192.168.107.114,0.998404
1,192.168.107.126,0.995649
9,192.168.107.58,0.965760
6,192.168.107.227,0.437545
7,192.168.107.234,0.436713
8,192.168.107.238,0.111088
4,192.168.107.223,0.109747
3,192.168.107.222,0.109633
5,192.168.107.226,0.109025


#### ip com aumento anomalo de trafico para os servidores da empresa
| dst_ip | ratio |
|------|-------|
| 192.168.107.205 | 1.023619 |
| 192.168.107.114 | 0.998404 |
| 192.168.107.126 | 0.995649 |
| 192.168.107.58  | 0.965760 |


In [63]:
servers_mean_ratios.to_csv("dst_ip_test.csv")


In [64]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_internal_test = test[test.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET  and  ipaddress.IPv4Address(x['src_ip']) in NET  ,axis=1)]

In [65]:
private_dst_internal_test_src_dst = private_dst_internal_test.groupby("src_ip")['dst_ip'].agg(set).reset_index()

In [66]:
private_dst_internal_test_src_dst.to_csv("test_coms_internal.csv")

In [67]:
private_dst_internal_normal = pd.read_csv("normal_coms_internal.csv")
private_dst_internal_test_src_dst = pd.read_csv("test_coms_internal.csv")

private_dst_internal_test_src_dst['dst_ip'] = private_dst_internal_test_src_dst['dst_ip'].apply(lambda x: set(x.replace("}","").replace("{","").replace(" ","").replace("\'","").split(",")))
private_dst_internal_normal['dst_ip'] = private_dst_internal_normal['dst_ip'].apply(lambda x: set(x.replace("}","").replace("{","").replace("\'","").replace(" ","").split(",")))


# Merge the two testFrames on src_ip
merged = pd.merge(private_dst_internal_test_src_dst,private_dst_internal_normal , on='src_ip', suffixes=('_test', '_normal'))

# Function to find dst_ip in test but not in normal
def find_diff(row):
    return list(set(row['dst_ip_test']).symmetric_difference(set(row['dst_ip_normal'])))

# Apply the function to each row
merged['unique_dst_ip'] = merged.apply(find_diff, axis=1)

# Filter out rows where there are no unique dst_ip values
unique_dst_ips = merged[['src_ip', 'unique_dst_ip']][merged['unique_dst_ip'].map(len) > 0]


In [68]:
unique_dst_ips.to_csv("ips_com_coms_novas_na_rede_interna.csv")

In [69]:
unique_dst_ips

,src_ip,unique_dst_ip
15,192.168.107.114,"[192.168.107.205, 192.168.107.58, 192.168.107...."
28,192.168.107.126,"[192.168.107.114, 192.168.107.205, 192.168.107..."
113,192.168.107.205,"[192.168.107.114, 192.168.107.58, 192.168.107...."
154,192.168.107.58,"[192.168.107.114, 192.168.107.205, 192.168.107..."
176,192.168.107.80,[192.168.107.222]


Botnet?
| row | src_ip | unique_dst_ip |
|----|-----|-------|
| 15 | 192.168.107.114 | "['192.168.107.205', '192.168.107.58', '192.168.107.126']" |
| 28 | 192.168.107.126 | "['192.168.107.114', '192.168.107.205', '192.168.107.58']" |
| 113 | 192.168.107.205 | "['192.168.107.114', '192.168.107.58', '192.168.107.126']" |
| 154 | 192.168.107.58 | "['192.168.107.114', '192.168.107.205', '192.168.107.126']" |
| 176 | 192.168.107.80 | ['192.168.107.222'] |


In [70]:
exterior_test = test[test.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) not in NET,axis=1)]



In [71]:
exterior_test_cc = exterior_test.copy()
exterior_test_cc.loc[:,'country_code'] = [gi.country_code_by_addr(addr) for addr in exterior_test_cc['dst_ip']]

grp_exterior_test = exterior_test.groupby(["proto","port"])['dst_ip'].apply(list)


In [72]:
exterior_test_cc.to_csv("test_cc.csv")

In [73]:
exterior_test_cc = pd.read_csv("test_cc.csv")

exterior_test_cc_grp = exterior_test_cc.groupby(['src_ip','country_code']).sum()

# Calculate the ratio
exterior_test_cc_grp['ratio'] = exterior_test_cc_grp['up_bytes'] /exterior_test_cc_grp['down_bytes']
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()
exterior_test_cc_grp=exterior_test_cc_grp.groupby(['src_ip','country_code'])['ratio'].mean().reset_index()

In [74]:
exterior_test_cc_grp.sort_values(by="ratio", ascending=False)

,src_ip,country_code,ratio
54,192.168.107.104,US,21.396821
1652,192.168.107.57,US,16.523158
1613,192.168.107.53,US,13.302599
2115,192.168.107.95,US,0.630343
2040,192.168.107.88,US,0.434094
...,...,...,...
1472,192.168.107.40,IE,0.049722
1131,192.168.107.20,BR,0.046604
1281,192.168.107.23,BR,0.043596
1305,192.168.107.24,SG,0.042752


 ### possiveis exflitracoes
 | idx |	src_ip | 	country_code | 	ratio |
 |-------|------|--------|---------|
| 54  | 	192.168.107.104 | 	US | 	21.396821 |
| 1652  | 	192.168.107.57  |	US  |	16.523158 |
| 1613  | 	192.168.107.53  |	US  |	13.302599 |

In [75]:
exterior_test_cc = pd.read_csv("test_cc.csv")

exterior_test_cc_grp = exterior_test_cc.groupby(['src_ip','dst_ip']).sum()

# Calculate the ratio
exterior_test_cc_grp['ratio'] = exterior_test_cc_grp['up_bytes'] /exterior_test_cc_grp['down_bytes']
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()
exterior_test_cc_grp=exterior_test_cc_grp.groupby(['src_ip','dst_ip'])['ratio'].mean().reset_index()

In [76]:
exterior_test_cc.loc[:,'ratio'] = exterior_test_cc['up_bytes']/exterior_test_cc['down_bytes']
exterior_test_cc_grp.sort_values(by="ratio", ascending=False)

,src_ip,dst_ip,ratio
18916,192.168.107.57,13.107.42.9,85.865247
18400,192.168.107.53,13.107.42.36,84.770370
450,192.168.107.104,13.107.42.9,82.773775
24575,192.168.107.95,104.244.42.65,28.816763
23757,192.168.107.88,104.244.42.129,4.035766
...,...,...,...
14736,192.168.107.23,138.2.246.153,0.020467
4190,192.168.107.134,68.221.144.226,0.020017
5632,192.168.107.145,205.251.243.79,0.019981
4677,192.168.107.139,8.25.195.7,0.019902


In [77]:
exterior_test_cc.groupby("dst_ip")["ratio"].mean().reset_index().sort_values(ascending=False, by='ratio').to_csv("dst_ip_ratio_test.csv")

In [78]:
exterior = pd.read_csv("dst_ip_ratio_normal.csv")


In [79]:
exterior.head()

,Unnamed: 0,src_ip,dst_ip,ratio
0,20076,192.168.107.55,52.144.197.179,0.256717
1,7204,192.168.107.154,52.126.224.7,0.248258
2,21794,192.168.107.69,157.55.103.170,0.246500
3,15533,192.168.107.25,52.93.60.50,0.238078
4,9221,192.168.107.170,150.222.214.120,0.235795


In [80]:
merged = pd.merge(exterior[["src_ip","ratio"]],exterior_test_cc_grp[["src_ip","ratio"]] , on="src_ip", suffixes=('_normal', '_test'))
merged1 = pd.merge(exterior[["dst_ip","ratio"]],exterior_test_cc_grp[["dst_ip","ratio"]] , on="dst_ip", suffixes=('_normal', '_test'))
merged2 = pd.merge(exterior[["src_ip","dst_ip","ratio"]],exterior_test_cc_grp[["src_ip","dst_ip","ratio"]] , on=["src_ip","dst_ip"], suffixes=('_normal', '_test'))



In [81]:
merged.head()

,src_ip,ratio_normal,ratio_test
0,192.168.107.55,0.256717,0.114543
1,192.168.107.55,0.256717,0.095188
2,192.168.107.55,0.256717,0.112875
3,192.168.107.55,0.256717,0.137678
4,192.168.107.55,0.256717,0.108709


In [82]:
merged1.head()

,dst_ip,ratio_normal,ratio_test
0,185.31.158.223,0.22166,0.119330
1,185.31.158.223,0.22166,0.111196
2,185.31.158.223,0.22166,0.111589
3,185.31.158.223,0.22166,0.111509
4,185.31.158.223,0.22166,0.118633


In [83]:
merged2.head()

,src_ip,dst_ip,ratio_normal,ratio_test
0,192.168.107.147,185.31.158.223,0.221660,0.117471
1,192.168.107.102,104.26.3.211,0.205241,0.115942
2,192.168.107.35,104.26.14.51,0.195635,0.129342
3,192.168.107.191,3.160.132.39,0.195297,0.116826
4,192.168.107.67,172.67.70.203,0.195274,0.070678


In [84]:
filtered_df = merged[merged['ratio_test'] >= 1.40 * merged['ratio_normal']]

print(filtered_df.sort_values(by="ratio_test", ascending=False))
filtered_df1 = merged1[merged1['ratio_test'] >= 1.40 * merged1['ratio_normal']]

print(filtered_df1.sort_values(by="ratio_test", ascending=False))
filtered_df2 = merged2[merged2['ratio_test'] >= 1.40 * merged2['ratio_normal']]

print(filtered_df2.sort_values(by="ratio_test", ascending=False))

                 src_ip  ratio_normal  ratio_test
2928940  192.168.107.57      0.104355   85.865247
2141626  192.168.107.57      0.118884   85.865247
2093824  192.168.107.57      0.119403   85.865247
1563803  192.168.107.57      0.123524   85.865247
2226981  192.168.107.57      0.117922   85.865247
...                 ...           ...         ...
3297228  192.168.107.68      0.062557    0.088029
3297135  192.168.107.68      0.062557    0.088011
3297683  192.168.107.76      0.061265    0.087820
3297699  192.168.107.76      0.061265    0.087811
3297778  192.168.107.76      0.061265    0.086849

[135954 rows x 3 columns]
                 dst_ip  ratio_normal  ratio_test
5479     104.244.42.129      0.168121    4.035766
351471   104.244.42.129      0.126637    4.035766
984148   104.244.42.129      0.121115    4.035766
1118056  104.244.42.129      0.119324    4.035766
735418   104.244.42.129      0.123102    4.035766
...                 ...           ...         ...
1413832  172.64.154.211

In [85]:
merged.loc[:,'ratio_increase'] = merged['ratio_test'] / merged['ratio_normal']
merged1.loc[:,'ratio_increase'] = merged1['ratio_test'] / merged1['ratio_normal']
merged2.loc[:,'ratio_increase'] = merged2['ratio_test'] / merged2['ratio_normal']

print(merged.sort_values(by="ratio_increase",ascending=False)[: ][:10])
print(merged1.sort_values(by="ratio_increase",ascending=False)[:][:10])
print(merged2.sort_values(by="ratio_increase",ascending=False)[:][:10])

                  src_ip  ratio_normal  ratio_test  ratio_increase
3288543   192.168.107.53      0.075497   84.770370     1122.837079
3285709   192.168.107.53      0.076907   84.770370     1102.250063
3269578   192.168.107.53      0.081329   84.770370     1042.313948
3269331   192.168.107.53      0.081506   84.770370     1040.056338
3272585  192.168.107.104      0.080529   82.773775     1027.881519
3259911   192.168.107.53      0.083578   84.770370     1014.265501
3260643  192.168.107.104      0.083322   82.773775      993.423071
3240065   192.168.107.53      0.086625   84.770370      978.590039
3226653   192.168.107.53      0.088009   84.770370      963.205405
3221999   192.168.107.53      0.088463   84.770370      958.259122
                 dst_ip  ratio_normal  ratio_test  ratio_increase
1390230  104.244.42.129      0.100314    4.035766       40.231258
1303641  104.244.42.129      0.113184    4.035766       35.656839
1276662  104.244.42.129      0.114735    4.035766       35.174691

In [86]:
data = pd.read_parquet("data7.parquet")
exterior_data = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) not in NET,axis=1)]
exterior_data_cc = exterior_data.copy()
exterior_data_cc.loc[:,'country_code'] = [gi.country_code_by_addr(addr) for addr in exterior_data_cc['dst_ip']]



In [87]:
exterior_data_cc.groupby("src_ip")["country_code"].agg(set).reset_index()

,src_ip,country_code
0,192.168.107.100,"{IN, NL, CN, GB, JP, US, BR, AU, PT, ES, NA, D..."
1,192.168.107.101,"{HK, IN, ID, NO, NL, JP, GB, US, BR, CA, BH, P..."
2,192.168.107.102,"{NL, GB, US, PT, ES, NA, DE}"
3,192.168.107.103,"{HK, NL, GB, US, BR, KR, IT, PT, ES, NA, DE, IE}"
4,192.168.107.104,"{IN, SE, NL, JP, GB, US, BR, IL, AU, ZA, PT, E..."
...,...,...
194,192.168.107.95,"{HK, NL, GB, US, BR, PT, ES, SG, NA, DE, IE}"
195,192.168.107.96,"{HK, IN, NL, JP, GB, CN, US, BR, AU, ZA, PT, E..."
196,192.168.107.97,"{NL, GB, US, BR, PT, ES, NA, DE}"
197,192.168.107.98,"{IN, SE, NL, JP, GB, US, BR, KR, FR, PT, ES, N..."


In [88]:
exterior_test_cc.groupby("src_ip")["country_code"].agg(set).reset_index()

,src_ip,country_code
0,192.168.107.100,"{HK, NL, JP, GB, BR, US, IL, IT, nan, BH, PT, ..."
1,192.168.107.101,"{SE, NL, GB, BR, US, nan, PT, SG, DE, IE}"
2,192.168.107.102,"{NO, NL, CN, GB, US, BR, IL, IT, nan, CH, BH, ..."
3,192.168.107.103,"{HK, IN, CN, NL, GB, US, BR, AU, nan, PT, ES, DE}"
4,192.168.107.104,"{ID, NL, JP, GB, BR, US, nan, PT, ES, DE}"
...,...,...
192,192.168.107.95,"{IN, NL, CN, GB, BR, US, nan, PT, ES, DE}"
193,192.168.107.96,"{HK, IN, ID, NL, CN, GB, BR, US, SA, KR, AU, n..."
194,192.168.107.97,"{HK, NL, JP, GB, US, BR, IT, nan, PT, ES, DE}"
195,192.168.107.98,"{NL, CN, GB, BR, US, AE, IL, IT, nan, PT, ES, DE}"


In [90]:
merged_cc = pd.merge(exterior_data_ccexterior_test_cc,, on="src_ip", suffixes=('_normal', '_test'))

MemoryError: Unable to allocate 18.0 GiB for an array with shape (2417710686,) and data type int64